In [1]:
import requests
import pandas as pd
from urllib.parse import unquote
import plotly.express as px

# URLs and headers
geturl = 'https://www.barchart.com/futures/quotes/CBV25/options/sep-25?futuresOptionsView=merged'
apiurl = 'https://www.barchart.com/proxies/core-api/v1/quotes/get'

getheaders = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
}

getpay = {
    'page': 'all'
}

s = requests.Session()
r = s.get(geturl, params=getpay, headers=getheaders)

headers = {
    'accept': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.barchart.com/futures/quotes/CBV25/options/sep-25?futuresOptionsView=merged',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
    'x-xsrf-token': unquote(unquote(s.cookies.get_dict()['XSRF-TOKEN']))
}

payload = {
    'fields': 'symbol,contractSymbol,lastPrice,priceChange,openPrice,highPrice,lowPrice,previousPrice,volume,openInterest,tradeTime,symbolCode,symbolType,hasOptions',
    'list': 'futures.contractInRoot',
    'root': 'CB',
    'meta': 'field.shortName,field.type,field.description',
    'hasOptions': 'true',
    'raw': '1'
}

r = s.get(apiurl, params=payload, headers=headers)
j = r.json()

# Convert JSON to DataFrame
df = pd.json_normalize(j['data'])  # Adjust the key based on the actual structure of your JSON
df = df.drop(0).reset_index(drop=True)
# Print DataFrame
print(df)
first_10_symbols = df['symbol'].iloc[:10].tolist()
print(first_10_symbols)


# Clean the 'lastPrice' column: remove non-numeric characters and convert to float
df['lastPrice'] = df['lastPrice'].str.replace('s', '', regex=False).astype(float)

# Plot using Plotly
fig = px.scatter(df, x='symbol', y='lastPrice', 
                 title='Forward Curve',
                 labels={'symbol': 'Contract Symbol', 'lastPrice': 'Last Price'},)

# Show the plot
fig.show()

   symbol   contractSymbol lastPrice priceChange openPrice highPrice lowPrice  \
0   CBU25  CBU25 (Sep '25)     68.65       +0.21     68.54     68.88    68.35   
1   CBV25  CBV25 (Oct '25)     67.81       +0.15     67.70     68.05    67.55   
2   CBX25  CBX25 (Nov '25)     67.18       +0.13     67.05     67.39    66.93   
3   CBZ25  CBZ25 (Dec '25)     66.79       +0.14     66.62     66.96    66.52   
4   CBF26  CBF26 (Jan '26)     66.55       +0.12     66.36     66.71    66.36   
..    ...              ...       ...         ...       ...       ...      ...   
74  CBX31  CBX31 (Nov '31)    66.41s       -0.29      0.00     66.41    66.41   
75  CBZ31  CBZ31 (Dec '31)    66.41s       -0.29      0.00     66.41    66.41   
76  CBF32  CBF32 (Jan '32)    66.41s       -0.29      0.00     66.41    66.41   
77  CBG32  CBG32 (Feb '32)    66.41s       -0.29      0.00     66.41    66.41   
78  CBH32  CBH32 (Mar '32)    66.41s       -0.29      0.00     66.41    66.41   

   previousPrice  volume op

In [30]:
import requests
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import datetime


# Base URL format
base_url = "https://www.barchart.com/futures/quotes/{}/options/"

# Headers to avoid 403 errors
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.google.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

# List to store extracted data
data = []

# Scrape data for each contract
for contract in first_10_symbols:
    url = base_url.format(contract)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        toolbar = soup.find('div', class_='row bc-options-toolbar__second-row')

        if toolbar:
            columns = toolbar.find_all('div', class_='column')
        

            days_to_expiry = columns[0].get_text(strip=True)
            implied_volatility = columns[1].get_text(strip=True)

            # Extract only numerical values
            expiration_date = days_to_expiry.split()[3][2:]
            days_to_expiry = int(days_to_expiry.split()[0])
            
            implied_volatility = float(implied_volatility.split(":")[1].replace("%", "").strip())

            # Get last price from df
            last_price = df.loc[df["symbol"] == contract, "lastPrice"].values[0]

            # Append data
            data.append([contract, last_price,expiration_date, days_to_expiry, implied_volatility])
        else:
            print(f"Could not find options toolbar for {contract}.")
    else:
        print(f"Failed to fetch data for {contract}. Status code: {response.status_code}")

# Create DataFrame
futures_df = pd.DataFrame(data, columns=["Contract", "Last Price","Expiration Date", "Options Days to Expiry", " Futures Implied Volatility"])


# Display the DataFrame
print(futures_df)


  Contract  Last Price Expiration Date  Options Days to Expiry  \
0    CBU25       68.65        07/28/25                       0   
1    CBV25       67.81        08/26/25                      29   
2    CBX25       67.18        09/25/25                      59   
3    CBZ25       66.79        10/28/25                      92   
4    CBF26       66.55        11/25/25                     120   
5    CBG26       66.48        12/23/25                     148   
6    CBH26       66.31        01/27/26                     183   
7    CBJ26       66.30        02/24/26                     211   
8    CBK26       66.38        03/26/26                     241   
9    CBM26       66.22        04/27/26                     273   

    Futures Implied Volatility  
0                        20.72  
1                        26.85  
2                        27.69  
3                        28.49  
4                        28.70  
5                        28.74  
6                        28.23  
7        

In [31]:
import requests
from urllib.parse import unquote
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
}

def main(url, symbols):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url[:25])
        req.headers.update(
            {'X-XSRF-TOKEN': unquote(r.cookies.get_dict()['XSRF-TOKEN'])})
        

        results = {}
        for i, symbol in enumerate(symbols, start=1):
            params = {
                "symbol": symbol,
                "list": "futures.options",
                "fields": "strike,openPrice,highPrice,lowPrice,lastPrice,priceChange,bidPrice,askPrice,volume,openInterest,premium,tradeTime,longSymbol,optionType,symbol",
                "orderBy": "strike",
                "orderDir": "asc",
                "meta": "field.shortName,field.description,field.type",
                "futureOptions": "true",
                "noPagination": "true",
                "showExpandLink": "false"
            }
            
            r = req.get(url, params=params).json()
            
            df = pd.DataFrame(r['data'])
            df['strike'] = df['strike'].astype(str).str.replace(r'[CP]', '', regex=True)
            df['volume'] = pd.to_numeric(df['volume'], errors='coerce').fillna(0).astype(int)
            df['openInterest'] = pd.to_numeric(df['openInterest'], errors='coerce').fillna(0).astype(int)
            df['lastPrice'] = df['lastPrice'].str.replace(r'[a-zA-Z]', '', regex=True).astype(float)

            
            
            calls_df = df[df['optionType'] == 'Call']
            puts_df = df[df['optionType'] == 'Put']
            
            results[symbol] = (calls_df, puts_df)

        return results
                

# Specify the URL and the output file path
url = 'https://www.barchart.com/proxies/core-api/v1/quotes/get?'



# Call the main function with the list of symbols and output file
results = main(url, first_10_symbols)



In [32]:
symbols = results.keys()
print(list(symbols))

for symbol, (calls_df, puts_df) in results.items():
    print(f"Symbol: {symbol}")
    
    print("DataFrame Columns:")
    print(calls_df.columns)  # Prints column names of calls DataFrame
    
    print("Calls DataFrame Sample:")
    print(calls_df.head())  # Prints first five rows of calls DataFrame
    
    print("Puts DataFrame Sample:")
    print(puts_df.head())  # Prints first five rows of puts DataFrame


['CBU25', 'CBV25', 'CBX25', 'CBZ25', 'CBF26', 'CBG26', 'CBH26', 'CBJ26', 'CBK26', 'CBM26']
Symbol: CBU25
DataFrame Columns:
Index(['strike', 'openPrice', 'highPrice', 'lowPrice', 'lastPrice',
       'priceChange', 'bidPrice', 'askPrice', 'volume', 'openInterest',
       'premium', 'tradeTime', 'longSymbol', 'optionType', 'symbol'],
      dtype='object')
Calls DataFrame Sample:
  strike openPrice highPrice lowPrice  lastPrice priceChange bidPrice  \
0  21.00       N/A     47.44      N/A      47.44       -0.74      N/A   
2  22.00       N/A     46.44      N/A      46.44       -0.74      N/A   
4  23.00       N/A     45.44      N/A      45.44       -0.74      N/A   
6  24.00       N/A     44.44      N/A      44.44       -0.74      N/A   
8  25.00       N/A     43.44      N/A      43.44       -0.74      N/A   

  askPrice  volume  openInterest    premium tradeTime longSymbol optionType  \
0      N/A       0             0  47,440.00  07/25/25  CBU5|210C       Call   
2      N/A       0     

In [33]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

for symbol, (calls_df, puts_df) in results.items():
    def clean_and_filter(df):
        df = df.copy()
        df['strike'] = pd.to_numeric(df['strike'], errors='coerce')
        df['openInterest'] = df['openInterest'].replace([np.inf, -np.inf], np.nan)
        df = df.dropna(subset=['strike', 'openInterest'])
        avg_oi = df['openInterest'].mean()
        # Filter to keep only rows with openInterest above average
        return df[df['openInterest'] > avg_oi].sort_values('strike')
    
    calls_filtered = clean_and_filter(calls_df)
    puts_filtered = clean_and_filter(puts_df)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=calls_filtered['strike'],
        y=calls_filtered['openInterest'],
        mode='markers',
        name='Calls Open Interest',
        marker=dict(color='blue', size=8, symbol='circle'),
        hovertemplate='Strike: %{x}<br>Open Interest: %{y}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=puts_filtered['strike'],
        y=puts_filtered['openInterest'],
        mode='markers',
        name='Puts Open Interest',
        marker=dict(color='red', size=8, symbol='x'),
        hovertemplate='Strike: %{x}<br>Open Interest: %{y}<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'Open Interest Scatter Plot Above Average for {symbol}',
        xaxis_title='Strike Price',
        yaxis_title='Open Interest',
        template='plotly_white',
        hovermode='closest'
    )
    
    fig.show()


In [36]:
# Get the current futures price (ATM)
atm_price = futures_df["Last Price"].iloc[0]  # Adjust if needed

for symbol, (calls_df, puts_df) in results.items():
    def clean_and_filter(df):
        df = df.copy()
        df['strike'] = pd.to_numeric(df['strike'], errors='coerce')
        df['volume'] = df['volume'].replace([np.inf, -np.inf], np.nan)
        df = df.dropna(subset=['strike', 'volume'])
        avg_vol = df['volume'].mean()
        return df[df['volume'] > avg_vol].sort_values('strike')
    
    calls_filtered = clean_and_filter(calls_df)
    puts_filtered = clean_and_filter(puts_df)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=calls_filtered['strike'],
        y=calls_filtered['volume'],
        mode='markers',
        name='Calls Volume',
        marker=dict(color='blue', size=8, symbol='circle'),
        hovertemplate='Strike: %{x}<br>Volume: %{y}<extra></extra>'
    ))
    
    fig.add_trace(go.Scatter(
        x=puts_filtered['strike'],
        y=puts_filtered['volume'],
        mode='markers',
        name='Puts Volume',
        marker=dict(color='red', size=8, symbol='x'),
        hovertemplate='Strike: %{x}<br>Volume: %{y}<extra></extra>'
    ))
    
    # Add ATM level vertical line
    fig.add_shape(
        dict(
            type="line",
            x0=atm_price,
            y0=0,
            x1=atm_price,
            y1=max(calls_filtered['volume'].max(), puts_filtered['volume'].max()) * 1.1,
            line=dict(color="green", width=2, dash="dash"),
            xref='x',
            yref='y'
        )
    )
    
    # Optional: add annotation for ATM line
    fig.add_annotation(
        x=atm_price,
        y=max(calls_filtered['volume'].max(), puts_filtered['volume'].max()) * 1.1,
        text="ATM",
        ax=0,
        ay=-40,
        font=dict(color="green")
    )
    
    fig.update_layout(
        title=f'Volume Scatter Plot Above Average for {symbol}',
        xaxis_title='Strike Price',
        yaxis_title='Volume',
        template='plotly_white',
        hovermode='closest'
    )
    
    fig.show()



In [38]:
import pandas as pd
import numpy as np

def find_volume_clusters(df, volume_col='volume', strike_col='strike', threshold_percentile=90, cluster_width=2):
    """
    Identify clusters of high volume strikes.
    
    Parameters:
    - df: DataFrame with strikes and volume
    - volume_col: column name for volume
    - strike_col: column name for strike prices
    - threshold_percentile: percentile above which volume is considered 'high'
    - cluster_width: max distance between strikes to consider part of the same cluster
    
    Returns:
    - List of clusters, each cluster is a list of strike prices
    """
    df = df.copy()
    df = df.dropna(subset=[strike_col, volume_col])
    
    # Find threshold volume
    threshold = np.percentile(df[volume_col], threshold_percentile)
    
    # Filter strikes with volume above threshold
    high_vol_df = df[df[volume_col] >= threshold].sort_values(strike_col)
    
    strikes = high_vol_df[strike_col].values
    
    clusters = []
    cluster = [strikes[0]]
    
    for i in range(1, len(strikes)):
        if strikes[i] - strikes[i-1] <= cluster_width:
            cluster.append(strikes[i])
        else:
            clusters.append(cluster)
            cluster = [strikes[i]]
    clusters.append(cluster)
    
    return clusters

# Example usage:
# Assuming you have combined calls and puts volume into one dataframe or analyze separately

# For calls
calls_clusters = find_volume_clusters(calls_filtered, volume_col='volume', strike_col='strike')

# For puts
puts_clusters = find_volume_clusters(puts_filtered, volume_col='volume', strike_col='strike')

print("Calls volume clusters (strike ranges):", calls_clusters)
print("Puts volume clusters (strike ranges):", puts_clusters)


Calls volume clusters (strike ranges): [[70.0]]
Puts volume clusters (strike ranges): [[45.0], [48.0]]


In [29]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Create one combined figure for all calls and one for all puts
fig_calls = go.Figure()
fig_puts = go.Figure()

for symbol, (calls_df, puts_df) in results.items():
    def clean_and_filter(df):
        df = df.copy()
        df['strike'] = pd.to_numeric(df['strike'], errors='coerce')
        df['openInterest'] = df['openInterest'].replace([np.inf, -np.inf], np.nan)
        df = df.dropna(subset=['strike', 'openInterest'])
        avg_oi = df['openInterest'].mean()
        return df[df['openInterest'] > avg_oi]

    calls_filtered = clean_and_filter(calls_df)
    puts_filtered = clean_and_filter(puts_df)

    # Add each symbol's calls to the combined calls figure
    fig_calls.add_trace(go.Histogram(
        x=calls_filtered['strike'],
        name=symbol,
        opacity=0.6,
        nbinsx=100
    ))

    # Add each symbol's puts to the combined puts figure
    fig_puts.add_trace(go.Histogram(
        x=puts_filtered['strike'],
        name=symbol,
        opacity=0.6,
        nbinsx=100
    ))

# Update layout for calls
fig_calls.update_layout(
    title='Calls Strike Price Distribution for Brent (OI > Avg) by Symbol',
    xaxis_title='Strike Price',
    yaxis_title='Frequency',
    template='plotly_white',
    barmode='overlay'
)

# Update layout for puts
fig_puts.update_layout(
    title='Puts Strike Price Distribution for Brent (OI > Avg) by Symbol',
    xaxis_title='Strike Price',
    yaxis_title='Frequency',
    template='plotly_white',
    barmode='overlay'
)

# Show figures
fig_calls.show()
fig_puts.show()


In [24]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select only the first, third, and sixth symbols
selected_keys = list(results.keys())
selected_symbols = [selected_keys[i] for i in [0, 2, 5] if i < len(selected_keys)]

heatmap_data = []
all_strikes = set()

for symbol in selected_symbols:
    calls_df, puts_df = results[symbol]

    def clean_and_filter(df):
        df = df.copy()
        df['strike'] = pd.to_numeric(df['strike'], errors='coerce')
        df['strike'] = np.floor(df['strike'])  # Round down to nearest integer
        df['openInterest'] = df['openInterest'].replace([np.inf, -np.inf], np.nan)
        df = df.dropna(subset=['strike', 'openInterest'])
        df = df[(df['strike'] >= 40) & (df['strike'] <= 100)]  # Limit to range 40–100
        avg_oi = df['openInterest'].mean()
        return df[df['openInterest'] > avg_oi]

    merged_df = pd.concat([clean_and_filter(calls_df), clean_and_filter(puts_df)])
    
    grouped = merged_df.groupby('strike')['openInterest'].sum().reset_index()
    grouped['symbol'] = symbol
    heatmap_data.append(grouped)

    all_strikes.update(grouped['strike'])

# Final pivoted DataFrame
heatmap_df = pd.concat(heatmap_data)
pivot_df = heatmap_df.pivot_table(
    index='strike',
    columns='symbol',
    values='openInterest',
    fill_value=0
)

# Ensure sorted and limited to range 40–100
pivot_df = pivot_df.sort_index()
pivot_df = pivot_df[(pivot_df.index >= 40) & (pivot_df.index <= 100)]
pivot_df = pivot_df[selected_symbols]

# Plot heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,
    y=pivot_df.index.astype(int),
    colorscale='Viridis',
    colorbar=dict(title='Open Interest'),
    hovertemplate='Symbol: %{x}<br>Strike: %{y}<br>OI: %{z}<extra></extra>'
))

fig.update_layout(
    title='Open Interest Heatmap (Strikes 40–100, Rounded, Above Avg)',
    xaxis_title='Symbol',
    yaxis_title='Strike Price',
    template='plotly_white'
)

fig.show()


In [37]:
import pandas as pd
import numpy as np

def find_volume_clusters(df, volume_col='volume', strike_col='strike', threshold_percentile=90, cluster_width=2):
    """
    Identify clusters of high volume strikes.
    
    Parameters:
    - df: DataFrame with strikes and volume
    - volume_col: column name for volume
    - strike_col: column name for strike prices
    - threshold_percentile: percentile above which volume is considered 'high'
    - cluster_width: max distance between strikes to consider part of the same cluster
    
    Returns:
    - List of clusters, each cluster is a list of strike prices
    """
    df = df.copy()
    df = df.dropna(subset=[strike_col, volume_col])
    
    # Find threshold volume
    threshold = np.percentile(df[volume_col], threshold_percentile)
    
    # Filter strikes with volume above threshold
    high_vol_df = df[df[volume_col] >= threshold].sort_values(strike_col)
    
    strikes = high_vol_df[strike_col].values
    
    clusters = []
    cluster = [strikes[0]]
    
    for i in range(1, len(strikes)):
        if strikes[i] - strikes[i-1] <= cluster_width:
            cluster.append(strikes[i])
        else:
            clusters.append(cluster)
            cluster = [strikes[i]]
    clusters.append(cluster)
    
    return clusters

# Example usage:
# Assuming you have combined calls and puts volume into one dataframe or analyze separately

# For calls
calls_clusters = find_volume_clusters(calls_filtered, volume_col='volume', strike_col='strike')

# For puts
puts_clusters = find_volume_clusters(puts_filtered, volume_col='volume', strike_col='strike')

print("Calls volume clusters (strike ranges):", calls_clusters)
print("Puts volume clusters (strike ranges):", puts_clusters)


Calls volume clusters (strike ranges): [[70.0]]
Puts volume clusters (strike ranges): [[45.0], [48.0]]
